# Spotipy - Retrieving playlist and track data from Spotify

## 1. Workplace Preparation

On Spotify's Developer website, a python library called **Spotipy** is recommended. I decided to utilise this library for my data collection from Spotify.

Library Source/Documentation: https://github.com/plamere/spotipy

In [4]:
# Imports the library for utilising Spotify's API by Python
!pip install spotipy


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.



In [5]:
# Token initialisation
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="7b1fa7a7eb25461f8d3a4a66e1966de5",
                                                           client_secret="cd8fa0032d964a1bbf6381b3b471d74e"))


In [180]:
import sys
import pprint
from pprint import pprint
import pandas as pd

## 2. Which Playlist?

First of all, I want to retrieve a list of featured playlists (Editor's Picks) on Spotify to see if there is a playlist that looks like it is worthy of investigation. These playlists usually contain popular tracks and are curated by Spotify's in-house team (Occhino, 2020). 

In [25]:
response = sp.featured_playlists()
print(response['message'])

while response:
    playlists = response['playlists']
    for i, item in enumerate(playlists['items']):
        print(playlists['offset'] + i, item['name'])

    if playlists['next']:
        response = sp.next(playlists)
    else:
        response = None

Editor's picks
0 New Music Friday
1 Feel Good Friday
2 RapCaviar
3 Main Stage
4 I Love My '90s Hip-Hop
5 Mood Booster
6 Dance Hits
7 Today's Top Hits
8 just hits
9 Dance Party
10 Happy 80s
11 young & free


The sixth result is called **"Mood Booster"**, which is highly related to my research objective, which is trying to analyse the mood of a song with sentimental analysis. I am curious about how the songs in this playlist are like.

To dig deeper into this playlist, I need to know the playlist's ID on Spotify. Unfortunately, the only feasible way to get a playlist's ID is through getting a user's current playlist. Here, I have to manually follow this playlist on Spotify and add it to my profile using my own Spotify account.

In [27]:
# Shows a user's playlists

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Whoops, need a username!")
    print("usage: python user_playlists.py [username]")
    sys.exit()

# Using my own username as input
playlists = sp.user_playlists("214hpvbrd65hxnr2sq2rmwmxa")

for playlist in playlists['items']:
    print(playlist['id'],playlist['name'])


37i9dQZF1DX3rxVfibe1L0 Mood Booster


We now get the Spotify ID of the playlist. With this result as the the parameter of our next methods, we will be able to retrieve more information about this the playlist.

## 3. Tracks in the Playlist

We will utilise the Spotify **playlist_tracks** method to get a fixed output of the tracks in a certain playlist.

In [194]:
# Retrieves the track names in the playlist

# Input the playlist ID we got in the last step
pl_id = '37i9dQZF1DX3rxVfibe1L0'
offset = 0

# Creates an empty list to store the track lists
playlist_content_list = []

while True:
    playlist_content = sp.playlist_items(pl_id,
                                 offset=offset,
                                 fields='items.track.name,total',
                                 additional_types=['track'])
    
    if len(playlist_content['items']) == 0:
        break
    
    pprint(playlist_content['items'])
    
    # Appends the content to the empty list I created earlier
    playlist_content_list.append(playlist_content['items'])
    offset = offset + len(playlist_content['items'])

    # Shows the length of the playlist
    print(offset, "/", playlist_content['total'])

[{'track': {'name': 'Little Bit of Love'}},
 {'track': {'name': 'Can I Get It'}},
 {'track': {'name': 'Dancing Feet (feat. DNCE)'}},
 {'track': {'name': 'Better Days (NEIKED x Mae Muller x Polo G)'}},
 {'track': {'name': "Let's Fall in Love for the Night"}},
 {'track': {'name': 'Meet Me At Our Spot'}},
 {'track': {'name': 'Heat Waves'}},
 {'track': {'name': 'When I’m Gone (with Katy Perry)'}},
 {'track': {'name': 'Glad You Exist'}},
 {'track': {'name': 'Wild (feat. Gary Clark Jr.)'}},
 {'track': {'name': 'Butterflies'}},
 {'track': {'name': 'Lil Bit'}},
 {'track': {'name': 'Overpass Graffiti'}},
 {'track': {'name': 'You (with Marshmello & Vance Joy)'}},
 {'track': {'name': 'The Bones - with Hozier'}},
 {'track': {'name': 'You Were Loved (with OneRepublic)'}},
 {'track': {'name': 'dancing in the kitchen'}},
 {'track': {'name': 'Love Again'}},
 {'track': {'name': 'Know Your Worth'}},
 {'track': {'name': 'Blueberry Eyes (feat. SUGA of BTS)'}},
 {'track': {'name': 'Heartbreak Anthem (with 

There are 76 songs in the playlist. I now need to clean it into a list that only contains indices and the track names for better future usability.

In [195]:
# The current track list 
playlist_content_list

[[{'track': {'name': 'Little Bit of Love'}},
  {'track': {'name': 'Can I Get It'}},
  {'track': {'name': 'Dancing Feet (feat. DNCE)'}},
  {'track': {'name': 'Better Days (NEIKED x Mae Muller x Polo G)'}},
  {'track': {'name': "Let's Fall in Love for the Night"}},
  {'track': {'name': 'Meet Me At Our Spot'}},
  {'track': {'name': 'Heat Waves'}},
  {'track': {'name': 'When I’m Gone (with Katy Perry)'}},
  {'track': {'name': 'Glad You Exist'}},
  {'track': {'name': 'Wild (feat. Gary Clark Jr.)'}},
  {'track': {'name': 'Butterflies'}},
  {'track': {'name': 'Lil Bit'}},
  {'track': {'name': 'Overpass Graffiti'}},
  {'track': {'name': 'You (with Marshmello & Vance Joy)'}},
  {'track': {'name': 'The Bones - with Hozier'}},
  {'track': {'name': 'You Were Loved (with OneRepublic)'}},
  {'track': {'name': 'dancing in the kitchen'}},
  {'track': {'name': 'Love Again'}},
  {'track': {'name': 'Know Your Worth'}},
  {'track': {'name': 'Blueberry Eyes (feat. SUGA of BTS)'}},
  {'track': {'name': 'Hea

In [196]:
# Removes the outter list
playlist_content_list = playlist_content_list[0]
playlist_content_list

[{'track': {'name': 'Little Bit of Love'}},
 {'track': {'name': 'Can I Get It'}},
 {'track': {'name': 'Dancing Feet (feat. DNCE)'}},
 {'track': {'name': 'Better Days (NEIKED x Mae Muller x Polo G)'}},
 {'track': {'name': "Let's Fall in Love for the Night"}},
 {'track': {'name': 'Meet Me At Our Spot'}},
 {'track': {'name': 'Heat Waves'}},
 {'track': {'name': 'When I’m Gone (with Katy Perry)'}},
 {'track': {'name': 'Glad You Exist'}},
 {'track': {'name': 'Wild (feat. Gary Clark Jr.)'}},
 {'track': {'name': 'Butterflies'}},
 {'track': {'name': 'Lil Bit'}},
 {'track': {'name': 'Overpass Graffiti'}},
 {'track': {'name': 'You (with Marshmello & Vance Joy)'}},
 {'track': {'name': 'The Bones - with Hozier'}},
 {'track': {'name': 'You Were Loved (with OneRepublic)'}},
 {'track': {'name': 'dancing in the kitchen'}},
 {'track': {'name': 'Love Again'}},
 {'track': {'name': 'Know Your Worth'}},
 {'track': {'name': 'Blueberry Eyes (feat. SUGA of BTS)'}},
 {'track': {'name': 'Heartbreak Anthem (with 

In [197]:
# For loops to retrieves the inner information
playlist_content_list = [info[name] 
                         for i in playlist_content_list 
                         for track,info in i.items() 
                         for name in info]
playlist_content_list

['Little Bit of Love',
 'Can I Get It',
 'Dancing Feet (feat. DNCE)',
 'Better Days (NEIKED x Mae Muller x Polo G)',
 "Let's Fall in Love for the Night",
 'Meet Me At Our Spot',
 'Heat Waves',
 'When I’m Gone (with Katy Perry)',
 'Glad You Exist',
 'Wild (feat. Gary Clark Jr.)',
 'Butterflies',
 'Lil Bit',
 'Overpass Graffiti',
 'You (with Marshmello & Vance Joy)',
 'The Bones - with Hozier',
 'You Were Loved (with OneRepublic)',
 'dancing in the kitchen',
 'Love Again',
 'Know Your Worth',
 'Blueberry Eyes (feat. SUGA of BTS)',
 'Heartbreak Anthem (with David Guetta & Little Mix)',
 'My Universe',
 'Wave of You',
 'seaside_demo',
 'Acapulco',
 'WHERE WE ARE',
 "Let's go to Hell",
 'Dandelions',
 'Chasing Stars (feat. James Bay)',
 'West Coast',
 'Shivers',
 'Where Are You Now',
 'Share That Love (feat. G-Eazy)',
 'Way Less Sad',
 'Make You Mine',
 'Cloudy Day',
 'Catching Feelings (feat. Six60)',
 'Big Energy',
 'Sunshine',
 'Lost',
 'A-O-K',
 'Levitating (feat. DaBaby)',
 'I AM WOMAN

We now get a list that only contains the song name.

In [148]:
# Retrieves the artist names of the tracks in the playlist

# Input the playlist ID we got in the last step
pl_id = '37i9dQZF1DX3rxVfibe1L0'
offset = 0

# Creates an empty list to store the track lists
playlist_id_list = []

while True:
    playlist_id = sp.playlist_items(pl_id,
                                 offset=offset,
                                 fields='items.track.id,total',
                                 additional_types=['track'])
    
    if len(playlist_id['items']) == 0:
        break
    
    pprint(playlist_id['items'])
    
    # Appends the content to the empty list I created earlier
    playlist_id_list.append(playlist_id['items'])
    offset = offset + len(playlist_id['items'])

    # Shows the length of the playlist
    print(offset, "/", playlist_id['total'])

[{'track': {'id': '78q4ESvMkPVJzHAV11LAGE'}},
 {'track': {'id': '6w8ZPYdnGajyfPddTWdthN'}},
 {'track': {'id': '4RAR8g8fZNB106ezUurnE0'}},
 {'track': {'id': '6f5ExP43esnvdKPddwKXJH'}},
 {'track': {'id': '7kQkmyoHCEqwe7QwDbkSXM'}},
 {'track': {'id': '07MDkzWARZaLEdKxo6yArG'}},
 {'track': {'id': '02MWAaffLxlfxAUY7c5dvx'}},
 {'track': {'id': '5902W4uHWzhtOff1UK7the'}},
 {'track': {'id': '472vIK1ldetTxRxG3ovaiY'}},
 {'track': {'id': '4rVW6XqAsSaf5vOwc8FREW'}},
 {'track': {'id': '7eQHxigpuDJjCG50JyzU8v'}},
 {'track': {'id': '0NmuYnjETG3u3qx0OmEJev'}},
 {'track': {'id': '4btFHqumCO31GksfuBLLv3'}},
 {'track': {'id': '1GkHyypTFkUf0QQKwYoXH4'}},
 {'track': {'id': '1yTTMcUhL7rtz08Dsgb7Qb'}},
 {'track': {'id': '4W1JavoraGzh83nluQHY6C'}},
 {'track': {'id': '0ohcCrxZkBfFbkuRPOZQZX'}},
 {'track': {'id': '1imMjt1YGNebtrtTAprKV7'}},
 {'track': {'id': '0TrPqhAMoaKUFLR7iYDokf'}},
 {'track': {'id': '5dn6QANKbf76pANGjMBida'}},
 {'track': {'id': '5K6Ssv4Z3zRvxt0P6EKUAP'}},
 {'track': {'id': '3FeVmId7tL5YN8B

In [150]:
# Same process to get a clean list that only contains song IDs
playlist_id_list
playlist_id_list = playlist_id_list[0]

playlist_id_list = [info[id] 
                         for i in playlist_id_list 
                         for track,info in i.items() 
                         for id in info]
playlist_id_list

['78q4ESvMkPVJzHAV11LAGE',
 '6w8ZPYdnGajyfPddTWdthN',
 '4RAR8g8fZNB106ezUurnE0',
 '6f5ExP43esnvdKPddwKXJH',
 '7kQkmyoHCEqwe7QwDbkSXM',
 '07MDkzWARZaLEdKxo6yArG',
 '02MWAaffLxlfxAUY7c5dvx',
 '5902W4uHWzhtOff1UK7the',
 '472vIK1ldetTxRxG3ovaiY',
 '4rVW6XqAsSaf5vOwc8FREW',
 '7eQHxigpuDJjCG50JyzU8v',
 '0NmuYnjETG3u3qx0OmEJev',
 '4btFHqumCO31GksfuBLLv3',
 '1GkHyypTFkUf0QQKwYoXH4',
 '1yTTMcUhL7rtz08Dsgb7Qb',
 '4W1JavoraGzh83nluQHY6C',
 '0ohcCrxZkBfFbkuRPOZQZX',
 '1imMjt1YGNebtrtTAprKV7',
 '0TrPqhAMoaKUFLR7iYDokf',
 '5dn6QANKbf76pANGjMBida',
 '5K6Ssv4Z3zRvxt0P6EKUAP',
 '3FeVmId7tL5YN8B7R3imoM',
 '5Ne1q9Hv3l2NHBA3Agt8WT',
 '73M0rMVx5CWE8M4uATSsto',
 '3eJH2nAjvNXdmPfBkALiPZ',
 '4MTmAFWHpvB9kPMSRgLFRp',
 '38XLUjlR84JEwK0SOvX77a',
 '2eAvDnpXP5W0cVtiI0PUxV',
 '6y6xhAgZjvxy5kR5rigpY3',
 '0sBJA2OCEECMs0HsdIQhvR',
 '6bQfNiqyCX7UaQSvVVGo4I',
 '3uUuGVFu1V7jTQL60S1r8z',
 '44l9nnCVvOQBbWG6tDViKl',
 '4jbtL4tjkqghUvJknUqU1s',
 '5iFwAOB2TFkPJk8sMlxP8g',
 '0mA7zotmg2ZFMRALljdZsS',
 '02VHspkXhhH1QCInRWWIfr',
 

We also need to know the artists of each track to enable further investigation. By using the track IDs as inputs, we can retrieve a number of attributes of the track by using Spotify's API, including the artist information.

In [163]:
# Using a random track as the input to see the result
track = sp.track('2LwH6T39A5IODRgPv9XitR')
pprint(track)

{'album': {'album_type': 'single',
           'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2ZmXexIJAD7PgABrj0qQRb'},
                        'href': 'https://api.spotify.com/v1/artists/2ZmXexIJAD7PgABrj0qQRb',
                        'id': '2ZmXexIJAD7PgABrj0qQRb',
                        'name': 'N.Flying',
                        'type': 'artist',
                        'uri': 'spotify:artist:2ZmXexIJAD7PgABrj0qQRb'}],
           'available_markets': ['AD',
                                 'AE',
                                 'AG',
                                 'AL',
                                 'AM',
                                 'AO',
                                 'AR',
                                 'AT',
                                 'AU',
                                 'AZ',
                                 'BA',
                                 'BB',
                                 'BD',
                                 'BE'

In [179]:
playlist_artist_list = []
for i in range(len(playlist_id_list)):
    track = sp.track(playlist_id_list[i])
    playlist_artist_list.append(track['album']['artists'][0]['name'])
playlist_artist_list                              

['Tom Grennan',
 'Adele',
 'Kygo',
 'NEIKED',
 'FINNEAS',
 'THE ANXIETY',
 'Glass Animals',
 'Alesso',
 'Dan + Shay',
 'John Legend',
 'MAX',
 'Nelly',
 'Ed Sheeran',
 'benny blanco',
 'Maren Morris',
 'Gryffin',
 'LANY',
 'Dua Lipa',
 'Khalid',
 'MAX',
 'Galantis',
 'Coldplay',
 'Surfaces',
 'SEB',
 'Jason Derulo',
 'The Lumineers',
 'Tai Verdes',
 'Ruth B.',
 'Alesso',
 'OneRepublic',
 'Ed Sheeran',
 'Lost Frequencies',
 'Lukas Graham',
 'AJR',
 'PUBLIC',
 'Tones And I',
 'Drax Project',
 'Latto',
 'OneRepublic',
 'Maroon 5',
 'Tai Verdes',
 'Dua Lipa',
 'Emmy Meli',
 'Gryffin',
 'Camila Cabello',
 'The Weeknd',
 'Niall Horan',
 'Marshmello',
 'Lil Nas X',
 'The Kid LAROI',
 'Vance Joy',
 'Andy Grammer',
 'P!nk',
 'MisterWives',
 'The Weeknd',
 'Tones And I',
 'Justin Bieber',
 'Quinn XCII',
 'Bazzi',
 'Kane Brown',
 'Marshmello',
 'Surfaces',
 'Walker Hayes',
 'BØRNS',
 'John Legend',
 'Harry Styles',
 'BANNERS',
 'Post Malone',
 'Joel Corry',
 'Dominic Fike',
 'Charlie Puth',
 'Pea

Now that we have three lists in hand, we can now create a dataframe out of them.

In [201]:
# Creates a Python dictionary first
spotify_df = {'track_name': playlist_content_list, 
              'spotify_id': playlist_id_list, 
              'artist_name': playlist_artist_list}
 
# Create a DataFrame from the dictionary
spotify_df = pd.DataFrame(spotify_df)
spotify_df

,track_name,spotify_id,artist_name
0,Little Bit of Love,78q4ESvMkPVJzHAV11LAGE,Tom Grennan
1,Can I Get It,6w8ZPYdnGajyfPddTWdthN,Adele
2,Dancing Feet (feat. DNCE),4RAR8g8fZNB106ezUurnE0,Kygo
3,Better Days (NEIKED x Mae Muller x Polo G),6f5ExP43esnvdKPddwKXJH,NEIKED
4,Let's Fall in Love for the Night,7kQkmyoHCEqwe7QwDbkSXM,FINNEAS
...,...,...,...
71,Mariposa,4ja2gzrNh9VNigzoXfmbwD,Peach Tree Rascals
72,Put Your Records On,1fah1uAs7HeTYDlNftKr3K,Ritt Momney
73,Message In A Bottle (Taylor's Version) (From T...,6PdCbJwSOeovMX7kfwiAxb,Taylor Swift
74,Summer of Love (Shawn Mendes & Tainy),0z8hI3OPS8ADPWtoCjjLl6,Shawn Mendes


I now have my first dataframe, which is about the tracks in the Spotify playlist, **Mood Booster** and the tracks' corresponding IDs and artists.

In [202]:
# Creates a CSV file for local storage of this Dataframe
spotify_df.to_csv('spotify_df.csv')

## References

Occhino, L., 2020. How to get your music featured on Spotify playlists. [online] Bandzoogle.com. Available at: <https://bandzoogle.com/blog/how-to-get-your-music-featured-on-spotify-playlists> [Accessed 8 April 2022].